In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import cross_val_score
import time
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical

#File path
#Please set the path to your own path
current = '/Users/apple/Desktop/5328ass2/datasets/'

# Create CNN model for FashionMNIST datasets
def create_model():
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(3, activation=tf.nn.softmax))
    model.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model
    
#Set label to one-hot catogory
def one_hot_embedding(Str, Yts):
    Str = to_categorical(Str, num_classes=3)
    Yts = to_categorical(Yts, num_classes=3)
    return Str, Yts


#Standardization, in order to increase the accuracy
def standard(Xtr, Xts):
    

    std = np.std(Xtr, keepdims=True)
    mean = np.mean(Xtr, keepdims=True)
    Xtr = (Xtr - mean) / std
    Xts = (Xts - mean) / std
    return Xtr, Xts


# Add the transition matrix to our model
def Addtran(T):

        T = K.constant(T)

        def loss(y_true, y_pred):
            y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
            y_pred = K.clip(y_pred, K.epsilon(), 1.0 - K.epsilon())
            return -K.sum(y_true * K.log(K.dot(y_pred, T)), axis=-1)

        return loss



valid_accuracy = np.zeros((10))
test_accuracy = np.zeros((10))

# Run for 10 times to get mean accuracy
for i in range(10):  

    # the choice of dataset
    fpath1 = os.path.join(current,'FashionMNIST0.5.npz')
    
    #If you want to use FashionMNIST0.6,  delete the '#'  
    #fpath1 = os.path.join(current,'FashionMNIST0.6.npz')
    
    # load data
    dataset = np.load(fpath1)

    Xtr_val = dataset['Xtr']
    Str_val = dataset['Str']
    Xts = dataset['Xts']
    Yts = dataset['Yts']
    
    #Standardization
    Xtr_val, Xts = standard(Xtr_val, Xts)

    # Random sample
    Str_val, Yts = one_hot_embedding(Str_val, Yts)
    a = np.argsort(np.random.random(Str_val.shape[0]))
    Xtr_val = Xtr_val[a]
    Str_val = Str_val[a]

    Total = len(Xtr_val)
    S = int(Total*0.8)
    
    # Choose 80% of data for trainning, the other 20% for valuating.
    Xtr_train = Xtr_val[:S]
    Str_train = Str_val[:S]
    Xtr_valid = Xtr_val[S:]
    Str_valid = Str_val[S:]

    Xtr_train = Xtr_train.reshape(Xtr_train.shape[0], 28, 28, 1)
    Xtr_valid = Xtr_valid.reshape(Xtr_valid.shape[0], 28, 28, 1)
    Xts = Xts.reshape(Xts.shape[0], 28, 28, 1)
    
    model = create_model()
    fpath = os.path.join(current,'CNNmodel.hdf5')
    
    # Provided transition matrix
    T1 = np.array([[0.5,0.3,0.2],[0.2,0.5,0.3],[0.3,0.2,0.5]])
    T2 = np.array([[0.4,0.3,0.3],[0.3,0.4,0.3],[0.3,0.3,0.4]])

    # The choice of transition matrix
    # If you want to use T2, change the parameter to 'T2'
    model.compile(loss=Addtran(T1),
                  optimizer=tf.keras.optimizers.Adam(lr=0.0005, decay=0.000001),
                  metrics=['accuracy'])   

    # Train the model and compare model in every epoch, save the best model to file
    checkpointer = ModelCheckpoint(filepath=fpath, verbose = 1, save_best_only=True)

    model.fit(Xtr_train,
              Str_train,
              epochs=30,
              batch_size=1024,
              validation_data=(Xtr_valid, Str_valid),
              callbacks=[checkpointer])
    time_1 = time.time()
    
    
    accuracy = model.evaluate(Xtr_valid,Str_valid)[1]
    print('The validation accuracy is :')
    print(accuracy)
    
    accuracy1 = model.evaluate(Xts,Yts)[1]
    print('The test accuracy is :')
    print(accuracy1)
    time_2 = time.time()
    valid_accuracy[i] = accuracy
    test_accuracy[i] = accuracy1
    
# Get the mean validation accuracy
print('The valid accuracy is :')
print(np.mean(valid_accuracy))
# Get the standard deviation of validation accuracy
print('The valid std is :')
print(valid_accuracy.std())
# Get the mean test accuracy
print('The test accuracy is :')
print(np.mean(test_accuracy))
# Get the standard deviation of test accuracy
print('The test std is :')
print(test_accuracy.std())


Train on 14400 samples, validate on 3600 samples
Epoch 1/30
12288/14400 [========================>.....] - ETA: 0s - loss: 1.0776 - acc: 0.4044
Epoch 00001: val_loss improved from inf to 1.05300, saving model to /Users/apple/Desktop/5328ass2/datasets/CNNmodel.hdf5


KeyboardInterrupt: 